<a href="https://colab.research.google.com/github/eflores5000/08MIAR_APR/blob/main/proyecto_final_APR_V15_en_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.2. Localizar entorno de trabajo: Google colab o local

In [1]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False


## 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
drive.flush_and_unmount()
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico  exists.

Colab: Changing directory to  /content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
/content/gdrive/My Drive/08_MIAR/actividades/proyecto practico
Archivos en el directorio: 
['dqn_models', 'checkpoints', 'dqn_05_weights.h5f.data-00000-of-00001', 'dqn_05_weights.h5f.index', 'dqn_06_weights.h5f.index', 'dqn_06_weights.h5f.data-00000-of-00001', 'dqn_07_weights.h5f.index', 'dqn_07_weights.h5f.data-00000-of-00001', 'dqn_08_weights.h5f.data-00000-of-00001', 'dqn_08_weights.h5f.index', '.ipynb_checkpoints', 'dqn_09_weights.h5f.index', 'dqn_09_weights.h5f.data-00000-of-00001', 'dqn_06_1_weights.h5f.data-00000-of-00001', 'dqn_06_1_weights.h5f.index', 'video', 'dqn_06_3_weights.h5f.index', 'dqn_06

## 1.4. Instalar librerías necesarias

In [3]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.12  #2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.05
  %pip install Keras==2.2.4
  %pip install tensorflow==2.1.0
  %pip install torch==2.0.1
  %pip install agents==1.4.0

  Cloning https://github.com/Kojoley/atari-py.git to /tmp/pip-req-build-vuzc9kv8
  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git /tmp/pip-req-build-vuzc9kv8
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py) ... done


## **PARTE 3**. Desarrollo y preguntas
Importar librerías

In [4]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, Permute
#from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint


#### Configuración base

In [5]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

height, width, channels = env.observation_space.shape
obs = env.reset()
print(height, width, channels)
print("Forma de la observación:", obs.shape)  # Debe ser (height, width, channels)
env.unwrapped.get_action_meanings()

210 160 3
Forma de la observación: (210, 160, 3)


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        #return processed_observation.astype('uint8')
        # Añadir canal para que sea (84, 84, 1)
        return np.expand_dims(processed_observation, axis=-1).astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)


## 1. EJFM Implementación de la red neuronal

In [7]:
def build_model(height, width, channels, actions):

    model = Sequential()
    #model.add(Permute((2, 3, 1), input_shape=(WINDOW_LENGTH, height, width)))
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(4,84,84,1))) # (4,height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model


In [8]:
#model = build_model(84, 84, 1, nb_actions)
model = build_model(height, width, channels, nb_actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 4, 20, 20, 32)     2080      
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 9, 9, 64)       32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 7, 7, 64)       36928     
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 512)               6423040   
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 6)                 1

## 2. EJFM Implementación de la solución DQN

In [9]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [10]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(
        EpsGreedyQPolicy(),
        attr='eps',
        value_max=1.0,
        value_min=0.1,
        value_test=0.2,
        nb_steps=5000)                   # Pasos de exploración

    memory = SequentialMemory(
        limit=5000,                       # Replay Buffer (Memoria de experiencias pasadas en total (s,a,r,s'))
        window_length=4)

    dqn = DQNAgent(model=model,
                   nb_actions=nb_actions,
                   policy=policy,
                   memory=memory,
                   processor=AtariProcessor(),
                   nb_steps_warmup=1000,    # Llena en buffer de pasos previos (s,a,r,s') sin evaluar aún.
                   gamma=0.99,
                   target_model_update=1000, # cada 10,000 steps actualiza la red objetivo
                   train_interval=4,          # Solo entrena cada n pasos de interacción con el entorno
                   delta_clip=1.0,            # Valor utilizado para recortar los errores TD (diferencia temporal) durante el entrenamiento para evitar fluctuaciones externas
                   enable_double_dqn=True,
                   enable_dueling_network=True,
                   dueling_type='avg',
                   batch_size=32,
                   )
    return dqn


In [11]:
from rl.callbacks import ModelIntervalCheckpoint

# Guardar el modelo cada 1000 pasos
checkpoint_callback = ModelIntervalCheckpoint(
    filepath='checkpoints/dqn_v15_weights_{step}.h5f',
    interval=1000,
    verbose=1
)

In [12]:
import os
os.makedirs("checkpoints", exist_ok=True)

In [13]:
from keras.optimizers import Adam

dqn = build_agent(model, nb_actions)
dqn.compile(Adam(learning_rate=0.0001, epsilon=0.001))


In [14]:
from datetime import datetime

print(f"⏱ Tiempo de inicio: {datetime.now().strftime('%H:%M:%S')}")

dqn.fit(env, nb_steps=100000, visualize=False, verbose=2,callbacks=[checkpoint_callback])

print(f"⏱ Tiempo de fin: {datetime.now().strftime('%H:%M:%S')}")

# Guardando modelo
dqn.save_weights('dqn_v15_weights.h5f')


⏱ Tiempo de inicio: 14:02:30
Training for 100000 steps ...


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   721/100000: episode: 1, duration: 6.084s, episode steps: 721, steps per second: 119, episode reward: 12.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 2.430 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
Step 1000: saving model to checkpoints/dqn_v15_weights_1000.h5f


/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  1236/100000: episode: 2, duration: 24.668s, episode steps: 515, steps per second:  21, episode reward:  3.000, mean reward:  0.006 [ 0.000,  1.000], mean action: 2.381 [0.000, 5.000],  loss: 0.005595, mean_q: 0.017218, mean_eps: 0.798760
  1872/100000: episode: 3, duration: 57.501s, episode steps: 636, steps per second:  11, episode reward: 10.000, mean reward:  0.016 [ 0.000,  1.000], mean action: 2.426 [0.000, 5.000],  loss: 0.004870, mean_q: 0.019405, mean_eps: 0.720640
Step 2000: saving model to checkpoints/dqn_v15_weights_2000.h5f
  2735/100000: episode: 4, duration: 77.297s, episode steps: 863, steps per second:  11, episode reward: 13.000, mean reward:  0.015 [ 0.000,  1.000], mean action: 2.015 [0.000, 5.000],  loss: 0.006710, mean_q: 0.023844, mean_eps: 0.585640
Step 3000: saving model to checkpoints/dqn_v15_weights_3000.h5f
  3817/100000: episode: 5, duration: 97.216s, episode steps: 1082, steps per second:  11, episode reward: 22.000, mean reward:  0.020 [ 0.000,  1.000], 

In [15]:
# Testing part to calculate the mean reward a los 500k
#weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
weights_filename = 'dqn_v15_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 18.000, steps: 781
Episode 2: reward: 12.000, steps: 638
Episode 3: reward: 15.000, steps: 684
Episode 4: reward: 14.000, steps: 794
Episode 5: reward: 6.000, steps: 367
Episode 6: reward: 10.000, steps: 510
Episode 7: reward: 13.000, steps: 666
Episode 8: reward: 3.000, steps: 347
Episode 9: reward: 7.000, steps: 390
Episode 10: reward: 19.000, steps: 1436
